In [194]:
# Archivos usados:
# modelo:'modelV4_BETO_uncased_fine_tune.pkl' va a ser el 9 despues! agregarlooo y borrar el 4
# dicc temas: 'Diccionario temas.xlsx'
# stopwords: 'Stop_words_sin_tilde1.xlsx'

#PROXIMOS PASOS:
# Agregar al dataset de sentimiento filas de PQRS provenientes de web y correo electrónico que no sean de agentes
# Relacionar las encuestas de PQRS a los PQRS para clasificar bien la temática
# periodo mensual acumuativo. cada dia, va a tomar de referencia los días del mes en curso para tomar las top frases y palabras
# limipar frases y palabras

################################# LIBRERIAS Y CONEXION ################################################

from sqlalchemy.engine import create_engine #Genera la conexión para generar queries
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertModel,BertTokenizer
from torch import nn
from nltk.util import ngrams
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import datetime as dt
from datetime import datetime 
import sqlalchemy as sa
from sqlalchemy.sql import text as sa_text


start_process=dt.datetime.today()


def conectionOracleDB():
    """ Configura la conexión a la bbdd proporcionada por Biwares(Nano). Devuleve la ruta de conexión como 'engine' """

    DIALECT = 'oracle'
    SQL_DRIVER = 'cx_oracle'
    USERNAME = 'EFECTY_VOC' #enter your username
    PASSWORD = 'Ef3cty2021' #enter your password
    HOST = 'insis.cl5u1tcuiscl.us-east-1.rds.amazonaws.com' #enter the oracle db host url
    PORT = 1521 # enter the oracle port number
    SERVICE = 'ORCL' # enter the oracle db service name
    ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE
    
    engine = create_engine(ENGINE_PATH_WIN_AUTH)
    
    return engine


print("Termina conexión",str(dt.datetime.today()-start_process))

######################################### QUERY ##############################################

engine=conectionOracleDB()

dia_inicial=str(dt.date.today()+ dt.timedelta(days=-2))
dia_final=str(dt.date.today())

# query="SELECT I.fecha,P.ENCUESTA, M.canal, M.MEDIO_INGRESO, I.INTERACCION_ID,C.ID ,C.TEMA_ID ,C.COMENTARIO, C.WRAPUP ,C.SUBTIPO ,C.FRASES ,C.SENTIMIENTO ,C.FECHA_PROCESO  FROM EFECTY_VOC.BT_COMENTARIOS C LEFT JOIN EFECTY_VOC.BT_INTERACCIONES I ON C.INTERACCION_ID=I.INTERACCION_ID LEFT JOIN EFECTY_VOC.LK_MEDIO_INGRESO M ON M.MEDIO_INGRESO_ID=I.MEDIO_INGRESO_ID LEFT JOIN LK_PREGUNTA P ON P.pregunta_id=I.pregunta_id where fecha>=date '"+dia_inicial+"' AND fecha<date'"+dia_final+"'"
query = """SELECT I.fecha,P.ENCUESTA, M.canal, M.MEDIO_INGRESO, 
            I.INTERACCION_ID,C.ID ,C.TEMA_ID ,C.COMENTARIO, 
            C.WRAPUP ,C.SUBTIPO ,C.FRASES ,C.SENTIMIENTO ,C.FECHA_PROCESO  
            FROM EFECTY_VOC.BT_COMENTARIOS C 
            LEFT JOIN EFECTY_VOC.BT_INTERACCIONES I ON C.INTERACCION_ID=I.INTERACCION_ID 
            LEFT JOIN EFECTY_VOC.LK_MEDIO_INGRESO M ON M.MEDIO_INGRESO_ID=I.MEDIO_INGRESO_ID 
            LEFT JOIN LK_PREGUNTA P ON P.pregunta_id=I.pregunta_id 
            where fecha>=TO_DATE('2021-11-21', 'YYYY-MM-DD') AND fecha<TO_DATE('2021-11-30', 'YYYY-MM-DD')"""
BT_inicial= pd.read_sql_query(query, conectionOracleDB()) #fetch first 100 row ONLY
BT_inicial_partida=BT_inicial[(BT_inicial['medio_ingreso']=='IVR')|(BT_inicial['medio_ingreso']=='MESSENGER')|(BT_inicial['canal']=='CUSTOMER SERVICE')]
BT_COMENTARIOS=BT_inicial[(BT_inicial['medio_ingreso']!='IVR')&(BT_inicial['medio_ingreso']!='MESSENGER')&(BT_inicial['canal']!='CUSTOMER SERVICE')]
# BT_COMENTARIOS_1 = pd.read_sql_query("SELECT I.fecha, P.ENCUESTA, M.canal, M.MEDIO_INGRESO, I.INTERACCION_ID,C.ID ,C.TEMA_ID ,C.COMENTARIO, C.WRAPUP ,C.SUBTIPO ,C.FRASES ,C.SENTIMIENTO ,C.FECHA_PROCESO  FROM EFECTY_VOC.BT_COMENTARIOS C LEFT JOIN EFECTY_VOC.BT_INTERACCIONES I ON C.INTERACCION_ID=I.INTERACCION_ID LEFT JOIN EFECTY_VOC.LK_MEDIO_INGRESO M ON M.MEDIO_INGRESO_ID=I.MEDIO_INGRESO_ID LEFT JOIN LK_PREGUNTA P ON P.pregunta_id=I.pregunta_id  where canal='PQRS' fetch first 100 row ONLY", engine)
# BT_COMENTARIOS_2 = pd.read_sql_query("SELECT I.fecha, P.ENCUESTA, M.canal, M.MEDIO_INGRESO, I.INTERACCION_ID,C.ID ,C.TEMA_ID ,C.COMENTARIO ,C.WRAPUP ,C.SUBTIPO ,C.FRASES ,C.SENTIMIENTO ,C.FECHA_PROCESO  FROM EFECTY_VOC.BT_COMENTARIOS C LEFT JOIN EFECTY_VOC.BT_INTERACCIONES I ON C.INTERACCION_ID=I.INTERACCION_ID LEFT JOIN EFECTY_VOC.LK_MEDIO_INGRESO M ON M.MEDIO_INGRESO_ID=I.MEDIO_INGRESO_ID LEFT JOIN LK_PREGUNTA P ON P.pregunta_id=I.pregunta_id  where canal='RRSS' fetch first 100 row ONLY", engine)
# BT_COMENTARIOS_3 = pd.read_sql_query("SELECT I.fecha, P.ENCUESTA, M.canal, M.MEDIO_INGRESO, I.INTERACCION_ID,C.ID ,C.TEMA_ID ,C.COMENTARIO ,C.WRAPUP ,C.SUBTIPO ,C.FRASES ,C.SENTIMIENTO ,C.FECHA_PROCESO  FROM EFECTY_VOC.BT_COMENTARIOS C LEFT JOIN EFECTY_VOC.BT_INTERACCIONES I ON C.INTERACCION_ID=I.INTERACCION_ID LEFT JOIN EFECTY_VOC.LK_MEDIO_INGRESO M ON M.MEDIO_INGRESO_ID=I.MEDIO_INGRESO_ID LEFT JOIN LK_PREGUNTA P ON P.pregunta_id=I.pregunta_id  where canal='ENCUESTA' fetch first 100 row ONLY", engine)
# BT_COMENTARIOS=pd.concat([BT_COMENTARIOS_3,BT_COMENTARIOS_2,BT_COMENTARIOS_1])
# BT_COMENTARIOS.reset_index(inplace=True)


print("Termina query",str(dt.datetime.today()-start_process))
print('La query mira desde 18 diciembre en adelante')

#################################################### TRABAJO SOBRE EL TEXTO #######################

#El comentario tal cual lo descargo, quedará con la marca "comentario inicial"
index_ini=BT_COMENTARIOS.index
BT_COMENTARIOS['comentario_inicial']=BT_COMENTARIOS['comentario']

#Lo paso a string y lo hago todo lower
BT_COMENTARIOS['comentario']=BT_COMENTARIOS['comentario'].astype(str)
index_ini=list(BT_COMENTARIOS.index)

BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x:x.lower())

#Modifico algunas cuestiones que afectan sobre todo a los PQRS: quito los /n, y los numeros a un mismo formato
BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r'[\n]{1,4}',' ',x)) #Mas que nada para las PQRS que tienen muchas lineas vacias, donde hay mucho \n textual
BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r"""[' ']{1,4}""",' ',x)) #CONTINUACIÓN DE LO DE ARRIBA
BT_COMENTARIOS.loc[index_ini,'comentario']=BT_COMENTARIOS['comentario'].apply(lambda x: re.sub(r"""[0-9]{1,10}""",'numero',x))

#Limpio comentarios de RRSS (no se tiene en cuenta medio de ingreso messenger)

BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentario'].apply(lambda x: re.findall("Cliente:\**(.*)\n\n",x) if re.match('.*\d\d/\d\d/\d\d\d\d.*',x) else x)
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub('\?','zzz',str(x)))
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub('\.','bbb',str(x)))
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub(',','qwqz',str(x)))
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub('[\W]',' ',str(x)))
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub('qwqz',',',str(x)))
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub('bbb','.',str(x)))
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub('zzz','?',str(x)))
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub('\s\s',' ',str(x)))
BT_COMENTARIOS.loc[index_ini,'comentarioRRSS']=BT_COMENTARIOS['comentarioRRSS'].apply(lambda x: re.sub('Imagen .*? [0-9-a-z]{10,30}','',str(x)))
index_rrss=BT_COMENTARIOS[(BT_COMENTARIOS['canal']=='RRSS')&(BT_COMENTARIOS['medio_ingreso']!='MESSENGER')].index
BT_COMENTARIOS.loc[index_rrss,'comentario']=BT_COMENTARIOS['comentarioRRSS']
BT_COMENTARIOS.drop(columns='comentarioRRSS',inplace=True)


print("Termina trabajo sobre texto",str(dt.datetime.today()-start_process))
############################# GENERACIÓN DEL SENTIMIENTO ##############################################

#cargo el tokenizador de beto
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

#donde voy a trabajar
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Vuelvo a definir la estructura del modelo donde voy a trabajar ( es la misma que en el entrenamiento)
class BERTSentimentClassifier(nn.Module):
  def __init__(self, n_classes):
    super(BERTSentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained("pytorch/")
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, cls_output = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask,
        return_dict=False
    )
    drop_output = self.drop(cls_output)
    output = self.linear(drop_output)
    return output

#Cargo el modelo
model = torch.load('modelV9_BETO_uncased_fine_tune.pkl', map_location=torch.device('cpu'))

#Defino el proceso del modelo para analizar el texto
def classifySentiment(review_text):
  encoding_review = tokenizer.encode_plus(
      review_text,
      truncation = True,
      add_special_tokens = True,
      return_token_type_ids = False,
      padding=True,
#       pad_to_max_length = True,
      return_attention_mask = True,
      return_tensors = 'pt'
      )
  
  input_ids = encoding_review['input_ids'].to(device)
  attention_mask = encoding_review['attention_mask'].to(device)
  output = model(input_ids, attention_mask)
  _, prediction = torch.max(output, dim=1) #El output es un vector de NCLASSES dimensiones (3). Aca, se va a quedar con el valor mas alto, quien será el elegido 
#   return prediction.tolist()[0],output.tolist()[0][0],output.tolist()[0][1],output.tolist()[0][2]
  return prediction.tolist()[0]

#Aplico el modelo

index_sen=BT_COMENTARIOS[((BT_COMENTARIOS['canal']=='PQRS')&(
                              (BT_COMENTARIOS['medio_ingreso']=='WEB')|
                              (BT_COMENTARIOS['medio_ingreso']=='CORREO ELECTRONICO')))|
               ((BT_COMENTARIOS['canal']=='RRSS')&
                              (BT_COMENTARIOS['medio_ingreso']!='MESSENGER'))|
                ((BT_COMENTARIOS['canal']=='ENCUESTA')&
                              (BT_COMENTARIOS['medio_ingreso']=='CRM'))].index

BT_COMENTARIOS.loc[index_sen,'sentimientoV4']=BT_COMENTARIOS.loc[index_sen,'comentario'].apply(lambda x:classifySentiment(x))

#Trabajo sobre los resultados
BT_COMENTARIOS['sentimiento']=BT_COMENTARIOS['sentimientoV4'].apply(lambda x: 0.5 if x==1 else x)
BT_COMENTARIOS['sentimiento']=BT_COMENTARIOS['sentimiento'].apply(lambda x: 1 if x==2 else x)
BT_COMENTARIOS.drop(columns=['sentimientoV4'],inplace=True)

print("Termina generación sentimiento",str(dt.datetime.today()-start_process))



####################################### GENERACION DE LOS TEMAS ###########################################

############ GENERACION DE TABLA TEMAS
#Extracción del diccionario
DicTemas = pd.read_excel('Diccionario temas.xlsx')

#Genero los diccionarios según el origen
Dic_Temas=dict(list(zip(DicTemas['SUBTIPO/WRAPUP/ENCUESTA'],DicTemas['TEMÁTICA'])))

df_temas=DicTemas[['SUBTIPO/WRAPUP/ENCUESTA','TEMÁTICA','sentimiento temática']]
dic_cort=df_temas.groupby(['TEMÁTICA','sentimiento temática']).count()
dic_cort.reset_index(inplace=True)
dic_cort=dic_cort[['TEMÁTICA','sentimiento temática']]
try:
    dic_cort.reset_index(inplace=True)
except:
    pass
dic_cort.rename(columns={'TEMÁTICA':'TEMA','sentimiento temática':'TEMA_SENTIMIENTO','index':'TEMA_ID'},inplace=True)
dic_cort['FECHA_CREACION'] = datetime.now()

dic_tema_id=dict(list(zip(dic_cort['TEMA'],dic_cort['TEMA_ID'])))
DicTemas['TEMA_ID']=DicTemas['TEMÁTICA'].apply(lambda x: int(dic_tema_id[x]))
Dic_Temas_subtipo=dict(list(zip(DicTemas['SUBTIPO/WRAPUP/ENCUESTA'][DicTemas['ORIGEN']=='Subtipo-PQRS'],DicTemas['TEMA_ID'][DicTemas['ORIGEN']=='Subtipo-PQRS'])))
Dic_Temas_wrapups=dict(list(zip(DicTemas['SUBTIPO/WRAPUP/ENCUESTA'][DicTemas['ORIGEN']=='wrapups'],DicTemas['TEMA_ID'][DicTemas['ORIGEN']=='wrapups'])))
Dic_Temas_enc=dict(list(zip(DicTemas['SUBTIPO/WRAPUP/ENCUESTA'][DicTemas['ORIGEN']=='Encuestas'],DicTemas['TEMA_ID'][DicTemas['ORIGEN']=='Encuestas'])))


BT_COMENTARIOS.reset_index(inplace=True)


#Paso a formato string las columnas donde voy a aplicar el diccionario
BT_COMENTARIOS['subtipo']=BT_COMENTARIOS['subtipo'].astype(str)
index_st=BT_COMENTARIOS[BT_COMENTARIOS['subtipo']=='None'].index
BT_COMENTARIOS.loc[index_st,'subtipo']=''
BT_COMENTARIOS['wrapup']=BT_COMENTARIOS['wrapup'].astype(str)
index_wr=BT_COMENTARIOS[BT_COMENTARIOS['wrapup']=='None'].index
BT_COMENTARIOS.loc[index_wr,'wrapup']=''
BT_COMENTARIOS['encuesta']=BT_COMENTARIOS['encuesta'].astype(str)

#Extracción de index para según origen
index_sub=BT_COMENTARIOS[BT_COMENTARIOS['subtipo']!=''].index
index_rrss=BT_COMENTARIOS[BT_COMENTARIOS['wrapup']!=''].index
index_enc=BT_COMENTARIOS[BT_COMENTARIOS['encuesta']!='None'].index

#Alpico los diccionarios según los index
BT_COMENTARIOS.loc[index_sub,'tema_id']=BT_COMENTARIOS['subtipo'].apply(lambda x: Dic_Temas_subtipo.get(x) if  Dic_Temas_subtipo.get(x)!='' else x)
BT_COMENTARIOS.loc[index_rrss,'tema_id']=BT_COMENTARIOS['wrapup'].apply(lambda x: Dic_Temas_wrapups.get(x) if  Dic_Temas_wrapups.get(x)!='' else x)
BT_COMENTARIOS.loc[index_enc,'tema_id']=BT_COMENTARIOS['encuesta'].apply(lambda x: Dic_Temas_enc.get(x) if  Dic_Temas_enc.get(x)!='None' else x)


print("Termina generación tema",str(dt.datetime.today()-start_process))
############################# EXTRACCION DE FRASES ######################################

query2="SELECT I.fecha,P.ENCUESTA, M.canal, M.MEDIO_INGRESO, I.INTERACCION_ID,C.ID ,C.TEMA_ID ,C.COMENTARIO, C.WRAPUP ,C.SUBTIPO ,C.FRASES ,C.SENTIMIENTO ,C.FECHA_PROCESO  FROM EFECTY_VOC.BT_COMENTARIOS C LEFT JOIN EFECTY_VOC.BT_INTERACCIONES I ON C.INTERACCION_ID=I.INTERACCION_ID LEFT JOIN EFECTY_VOC.LK_MEDIO_INGRESO M ON M.MEDIO_INGRESO_ID=I.MEDIO_INGRESO_ID LEFT JOIN LK_PREGUNTA P ON P.pregunta_id=I.pregunta_id where fecha>=date '"+dia_inicial+"' AND fecha<date'"+dia_final+"'"

# #Trabajo sobre un subdataframe para extracción de frases. No me interesa extraer frases de comentarios que no son del cliente
BT_COMENTARIOS_FRASES=BT_COMENTARIOS.copy()

# mes del comentario
# ver si las palabras y frases estan dentro de la tabla de bt_frases y bt_words del mes! agegarlo al proceso de limipez

#genero columna de mes
BT_COMENTARIOS['mes']=BT_COMENTARIOS['fecha'].apply(lambda x: x.month)
BT_COMENTARIOS[BT_COMENTARIOS['mes']==dt.date.today().month]


#Filtro mes actual y categorias que aplican
BT_COMENTARIOS_FRASES=BT_COMENTARIOS[
#                             (BT_COMENTARIOS['mes']==dt.date.today().month)&
                              (
                              ((BT_COMENTARIOS['canal']=='PQRS')&
                                  ((BT_COMENTARIOS['medio_ingreso']=='WEB')|
                                  (BT_COMENTARIOS['medio_ingreso']=='CORREO ELECTRONICO')))|
                              ((BT_COMENTARIOS['canal']=='RRSS')&
                                  (BT_COMENTARIOS['medio_ingreso']!='MESSENGER'))|
                              ((BT_COMENTARIOS['canal']=='ENCUESTA')&
                                  (BT_COMENTARIOS['medio_ingreso']=='CRM'))
                              )]

print('frases y palabras solo mira mes actual, y lo filtrado arriba')

#Genero 3 bases. Se van a extraer las frases mas comunes para cada grupo
BT_COMENTARIOS_FRASES_RRSS=BT_COMENTARIOS_FRASES[BT_COMENTARIOS_FRASES['canal']=='RRSS']
BT_COMENTARIOS_FRASES_ENCUESTA=BT_COMENTARIOS_FRASES[BT_COMENTARIOS_FRASES['canal']=='ENCUESTA']
BT_COMENTARIOS_FRASES_PQRS=BT_COMENTARIOS_FRASES[BT_COMENTARIOS_FRASES['canal']=='PQRS']


#Importo stopwords
stop_words = (pd.read_excel('Stop_words_sin_tilde.xlsx')) 
stop_words=list(stop_words['stopwords']) 

#Defino las frases que no me aportan valor
Elim_Fras_PQRS=['por favor','gracias','quedamos atentos','feliz','cordialmente','quedo atento',' c ',' com ',' co ',
'url','post','none','numero','hola','te informamos','efecty bienvenido','cliente htt','api',
'imagen enviada','puresocial','email','tratamiento de datos','acepta politica','twitter','pm','am',
'siguientes datos','nombres','por favor confirma','htt','www','facebook','efectyoficial','twitterefectyoficial','efecty'
 'cliente buenas taredes','hotmail','gmail','usuario se comunica','usuaria se comunica','confronta exitosa','usuario solicita información',
 'usuaria solicita información','se informa que','se le indica correo','se realiza','se solicita','se genera certificación de pagos',
 'se envía la solicitud al correo','por parte del usuariose envía','se genera certificación de pagos','del proyecto','se informa',
 'se brinda','usuaria','solicita información','se informa que','usuario solicita','usuaria solicita','retroalimentacion','se toma solicitud',
 'solicita','usuaria se comunica para manifestar inconformidad','usuario se comunica','se indica','se informa','cédula','celular','solicitud',
 'cajero:','punto:','correo:','errada:','correc:','radicado:','nombre:','cc:','correo:','telefono:','direccion:','radicado:','os:',
 'ps','usuario solicita','pago realizado','buenas tardes','datos personales','respuesta al correo electronico','correo electronico se indica','usuaria solicita',
 'adjunta pantallazo','al correo sac','se lee articulo','se adjunta','pantallazo de correo','confronta no exitoso','usuario autoriza','buenas tardes',
 'cordial saludo','correo electronico','dias habiles','buenos dias','respuesta al correo','autoriza respuesta','el dia de hoy','tiempo de gestion de','quedo atenta','muchas gracias','articulo de la ley','autoriza tiempo',
 'ley','buen dia','pronta respuesta','cedula','numero de referencia','hasta la fecha','el nombre del titular']


Elim_Fras_ENCUESTA=['por favor','gracias','quedamos atentos','feliz','cordialmente','quedo atento',' c ',' com ',' co ',
'url','post','none','numero','hola','te informamos','efecty bienvenido','cliente htt','api',
'imagen enviada','puresocial','email','tratamiento de datos','acepta politica','twitter','pm','am',
'siguientes datos','nombres','por favor confirma','htt','www','facebook','efectyoficial','twitterefectyoficial','efecty'
 'cliente buenas taredes','hotmail','gmail','usuario se comunica','usuaria se comunica','confronta exitosa','usuario solicita información',
 'usuaria solicita información','se informa que','se le indica correo','se realiza','se solicita','se genera certificación de pagos',
 'se envía la solicitud al correo','por parte del usuariose envía','se genera certificación de pagos','del proyecto','se informa',
 'se brinda','usuaria','solicita información','se informa que','usuario solicita','usuaria solicita','retroalimentacion','se toma solicitud',
 'solicita','usuaria se comunica para manifestar inconformidad','usuario se comunica','se indica','se informa','cédula','celular','solicitud',
 'cajero:','punto:','correo:','errada:','correc:','radicado:','nombre:','cc:','correo:','telefono:','direccion:','radicado:','os:',
 'ps','usuario solicita','pago realizado','buenas tardes','datos personales','respuesta al correo electronico','correo electronico se indica','usuaria solicita',
 'adjunta pantallazo','al correo sac','se lee articulo','se adjunta','pantallazo de correo','confronta no exitoso','usuario autoriza','buenas tardes',
 'cordial saludo','correo electronico','dias habiles','buenos dias','respuesta al correo','autoriza respuesta','el dia de hoy','tiempo de gestion de','quedo atenta','muchas gracias','articulo de la ley','autoriza tiempo',
 'ley','buen dia','pronta respuesta','cedula','numero de referencia','hasta la fecha','el nombre del titular']

Elim_Fras_RRSS=['por favor','gracias','quedamos atentos','feliz','cordialmente','quedo atento',' c ',' com ',' co ',
'url','post','none','numero','hola','te informamos','efecty bienvenido','cliente htt','api',
'imagen enviada','puresocial','email','tratamiento de datos','acepta politica','twitter','pm','am',
'siguientes datos','nombres','por favor confirma','htt','www','facebook','efectyoficial','twitterefectyoficial','efecty'
 'cliente buenas taredes','hotmail','gmail','usuario se comunica','usuaria se comunica','confronta exitosa','usuario solicita información',
 'usuaria solicita información','se informa que','se le indica correo','se realiza','se solicita','se genera certificación de pagos',
 'se envía la solicitud al correo','por parte del usuariose envía','se genera certificación de pagos','del proyecto','se informa',
 'se brinda','usuaria','solicita información','se informa que','usuario solicita','usuaria solicita','retroalimentacion','se toma solicitud',
 'solicita','usuaria se comunica para manifestar inconformidad','usuario se comunica','se indica','se informa','cédula','celular','solicitud',
 'cajero:','punto:','correo:','errada:','correc:','radicado:','nombre:','cc:','correo:','telefono:','direccion:','radicado:','os:',
 'ps','usuario solicita','pago realizado','buenas tardes','datos personales','respuesta al correo electronico','correo electronico se indica','usuaria solicita',
 'adjunta pantallazo','al correo sac','se lee articulo','se adjunta','pantallazo de correo','confronta no exitoso','usuario autoriza','buenas tardes',
 'cordial saludo','correo electronico','dias habiles','buenos dias','respuesta al correo','autoriza respuesta','el dia de hoy','tiempo de gestion de','quedo atenta','muchas gracias','articulo de la ley','autoriza tiempo',
 'ley','buen dia','pronta respuesta','cedula','numero de referencia','hasta la fecha','el nombre del titular']


qty_sentences=40

def limp_comentarios(df,col1):
    df[col1]=df[col1].apply(lambda x: re.sub('[\W]',' ',str(x))) #elimino todo lo que no es numero o letra
    df[col1]=df[col1].apply(lambda x: re.sub('\s\s',' ',str(x))) #elimino dbole espacio generado por lo de arriba
    df[col1]=df[col1].apply(lambda x: re.sub('á','a',str(x)))
    df[col1]=df[col1].apply(lambda x: re.sub('é','e',str(x)))
    df[col1]=df[col1].apply(lambda x: re.sub('í','i',str(x)))
    df[col1]=df[col1].apply(lambda x: re.sub('ó','o',str(x)))
    df[col1]=df[col1].apply(lambda x: re.sub('ú','u',str(x)))
    df[col1]=df[col1].apply(lambda x: re.sub(r"\wW]"," ",x))
    df[col1]=df[col1].apply(lambda x: re.sub(r"""[\n\t]"""," ",x))
    df[col1]=df[col1].apply(lambda x: re.sub("""[-*\_¡!@#$:).;–,¿?&'_>/]{1,4}""","",x))
    df[col1]=df[col1].apply(lambda x: re.sub(r"""[0-9]""","",x))

limp_comentarios(BT_COMENTARIOS_FRASES_RRSS,'comentario')
limp_comentarios(BT_COMENTARIOS_FRASES_PQRS,'comentario')
limp_comentarios(BT_COMENTARIOS_FRASES_ENCUESTA,'comentario')

#Elimino las frases que no aportan valor
def Elimifrases(fras_elim,df,column):
    for f in fras_elim:   
        df[column]=df[column].apply(lambda x: re.sub(f,'',str(x)))

Elimifrases(Elim_Fras_RRSS,BT_COMENTARIOS_FRASES_RRSS,'comentario')
Elimifrases(Elim_Fras_PQRS,BT_COMENTARIOS_FRASES_PQRS,'comentario')
Elimifrases(Elim_Fras_ENCUESTA,BT_COMENTARIOS_FRASES_ENCUESTA,'comentario')


def tokenizar(df,column):
    pa_token=re.sub("""[,']""",'',str(list(df[column])))
    tokenized_word=word_tokenize(pa_token)
    return tokenized_word

tokRRSS=tokenizar(BT_COMENTARIOS_FRASES_RRSS,'comentario')
tokPQRS=tokenizar(BT_COMENTARIOS_FRASES_PQRS,'comentario')
tokENCUESTA=tokenizar(BT_COMENTARIOS_FRASES_ENCUESTA,'comentario')
toTOT=tokRRSS+tokPQRS+tokENCUESTA

##desde aca, revisar

def gen_frases(df,qty_words_ngram,max_stopwords,tok,qty_most_common,col):
    global Top_words
    Top_words=[]
    Encuestas2=pd.DataFrame()
    ongrams=ngrams(tok,qty_words_ngram)
    ongrams=list(zip(ongrams))
    ongrams=pd.DataFrame(ongrams)
    ongrams['r']=0
    for i in range(0,qty_words_ngram):
        ongrams['val'+str(i)]=ongrams[0].apply(lambda x: x[i])
        ongrams['w'+str(i)]=0
        index_on_val=ongrams[ongrams['val'+str(i)].isin(stop_words)].index
        ongrams.loc[index_on_val,'w'+str(i)]=1
        ongrams['r']=ongrams['r']+ongrams['w'+str(i)]

    ongrams=ongrams[(ongrams['r']<=max_stopwords)&(ongrams['w'+str(qty_words_ngram-1)]==0)] #La ultima condicion es para que no termine con un stopword
    ongrams_=[]
    ongrams.reset_index(inplace=True)

    for i in range(len(ongrams[0])):
        ongrams_.append(re.sub("""[¡!@)#($:.;,¿?&'_]""","",str(ongrams[0][i]))) #Agregar nan


    fdist = FreqDist((ongrams_))

    df['frases']=''
    for i in range(1,qty_most_common-1):
        try:
            index_frases=df[(df[col].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']=='')].index
            index_frases_duplicadas=df[(df[col].str.contains(str(fdist.most_common(qty_most_common)[i][0])))&(df['frases']!='')].index
    #Si, veo que esta en el top, pero que no está en las frases self.dataframe['Descripción'], es porque son 2 palabras que entre medio tenian una frase que 
    # se eliminó y queda un espacio grande entre medio que te lo considera dentro del top, pero cuando lo busca no lo encuentra,
    # porque no existe tal cual, sino con un espacio de por medio y una frase sin sentido.
            df.loc[index_frases,'frases']=fdist.most_common(qty_most_common)[i][0]
            Top_words.append(fdist.most_common(qty_most_common)[i][0])
            aux2=df.loc[index_frases_duplicadas,:]
            aux2.loc[:,'frases']=fdist.most_common(qty_most_common)[i][0]
            Encuestas2=pd.concat([aux2,Encuestas2])
        except:
            pass
    try:
        Encuestas2.drop(columns=['level_0'])
        Encuestas2.reset_index(inplace=True)
    except:
        pass
#     Encuestas2.drop(columns=['level_0'])
    df=pd.concat([df,Encuestas2])
    try:
        Encuestas2.drop(columns=['level_0'])
        Encuestas2.reset_index(inplace=True)
    except:
        pass
    Encuestas_con_frases=df[df['frases']!='']
#     self.dataframe['frases']=self.dataframe['frases'].apply(lambda x: x.upper())
#     Top_words=fdist.most_common(self.qty_most_common)

    return df,Top_words


#Extracción de frases segun cantidad de palabras y stopwords + generación de una columna en el Df
#Guardo las top frases en top_2_palabras,
gen_frases(BT_COMENTARIOS_FRASES,2,0,toTOT,qty_sentences,'comentario')
R_frases2=BT_COMENTARIOS_FRASES[BT_COMENTARIOS_FRASES['frases']!='']
top_2_palabras=pd.DataFrame(Top_words)

gen_frases(BT_COMENTARIOS_FRASES,3,1,toTOT,qty_sentences,'comentario')
R_frases3=BT_COMENTARIOS_FRASES[BT_COMENTARIOS_FRASES['frases']!='']
top_3_palabras=pd.DataFrame(Top_words)

gen_frases(BT_COMENTARIOS_FRASES,4,2,toTOT,qty_sentences,'comentario')
R_frases4=BT_COMENTARIOS_FRASES[BT_COMENTARIOS_FRASES['frases']!='']
top_4_palabras=pd.DataFrame(Top_words)


#Concatener los resultados
Resultado=pd.concat([BT_COMENTARIOS_FRASES,R_frases2])
Resultado=pd.concat([Resultado,R_frases3])
Resultado=pd.concat([Resultado,R_frases4])
try:
    Resultado.reset_index(inplace=True)
except:
    pass



## y aca vendria la limpieza de abajo, que falta trabajar. Por ahora,
# solo esta aplicado a Resultados, que solo esta la base de ENCUESTAS. HABRIA Q REPETIR LO MISMO A PQRS Y RRSS


eliminar=[]
index_eliminar_2=[]
index_eliminar_3=[]
index_eliminar_4=[]
top_2_palabras_n=list(top_2_palabras[0])
top_3_palabras_n=list(top_3_palabras[0])
top_4_palabras_n=list(top_4_palabras[0])
re.findall('[\w]{1,15}',top_4_palabras[0][6])

for j in range(0,len(top_4_palabras)-1): 
    #Pensado para qty_most_common(30) palabras mas comunes de 4 palabras. Compara contra las de 4 y las de 3
    qs=re.findall('[\w]{1,15}',top_4_palabras[0][j])
    # 4 palabras solo va a ir contra 3  
    for i in range(0,len(qs)-1):
                    for p1 in range(0,len(top_4_palabras)-1):
#Mira si hay 2 palabras del top4 repetidas dentro de los 4grams, si es asi lo elimina
                        if(((qs[i]+' '+qs[i+1]) in top_4_palabras[0][p1])&(p1!=j)):
                            eliminar.append(top_4_palabras[0][p1])
                            top_4_palabras_n[p1]=top_4_palabras_n[p1]
                            index_eliminar_4.append(p1)
                        if((qs[i]+' '+qs[i+1]) in top_3_palabras[0][p1]):
                            eliminar.append(top_3_palabras[0][p1])
                            top_3_palabras_n[p1]=top_3_palabras_n[j]
                            index_eliminar_3.append(p1)
                            
#Mira si hay 2 palabras segúidas del top4 repetidas dentro de los 3grams, si es asi lo elimina
                        if((qs[i]+' '+qs[i+1]) in top_3_palabras[0][p1]):
                            eliminar.append(top_3_palabras[0][p1])
                            top_3_palabras_n[p1]=top_3_palabras_n[j]
                            index_eliminar_3.append(p1)

#Mira si hay 2 palabras del top4 segúidas repetidas dentro de los 2grams, si es asi lo elimina
                        if((qs[i]+' '+qs[i+1]) in top_2_palabras[0][p1]):
                            eliminar.append(top_2_palabras[0][p1])
                            index_eliminar_2.append(p1)
                            top_2_palabras_n[p1]= top_2_palabras_n[j]
                    
for j in range(0,len(top_3_palabras)-1): 
    qs3=re.findall('[\w]{1,15}',top_3_palabras[0][j])
    # 3 palabras solo va a ir contra 2
    for i in range(0,len(qs3)-1):
        for p1 in range(0,len(top_3_palabras)-1):
            #Mira si hay 2 palabras del top3 repetidas en top 3, si es asi lo elimina
            if(((qs3[i]+' '+qs3[i+1]) in top_3_palabras[0][p1])&(p1!=j)):
                eliminar.append(top_3_palabras[0][p1])
                index_eliminar_3.append(p1)
                top_3_palabras_n[p1]=top_3_palabras_n[j]

 #Mira si hay 2 palabras del top3 en top2, si es asi lo elimina
            if((qs3[i]+' '+qs3[i+1]) in top_2_palabras[0][p1]):
                eliminar.append(top_2_palabras[0][p1])
                index_eliminar_2.append(p1)
                top_2_palabras_n[p1]=top_2_palabras_n[j]
last_frases=top_2_palabras_n+top_3_palabras_n+top_4_palabras_n
 
    
#Genero lista de reducción de frases que contienen mismas palabras a su unidad mas pequeña
# last_frases=frases.LimpiezaFrases(top_2_palabras,top_3_palabras,top_4_palabras)
Resultado.drop_duplicates(inplace=True)

#Modifico las frases con excedentes por las mas pequeñas
for b in range(0,len(last_frases)):
    index_a_modificar=Resultado[Resultado['frases'].str.contains(last_frases[b])].index
    Resultado.loc[index_a_modificar,'frases']=last_frases[b] 


Resultado.drop_duplicates(inplace=True)

#Modifico las frases con excedentes por las mas pequeñas
for b in range(0,len(last_frases)):
    index_a_modificar=Resultado[Resultado['frases'].str.contains(last_frases[b])].index
    Resultado.loc[index_a_modificar,'frases']=last_frases[b] 


Resultado.drop_duplicates(inplace=True)
# Resultado[Resultado['frases']!=''].groupby('frases').count().sort_values(by='index',ascending=False).head(50)[['index','id']]
Resultado=Resultado[Resultado['frases']!='']
df_sentence=Resultado[['id','frases']]


print("Termina generación de frase",str(dt.datetime.today()-start_process))

############################# EXTRACCION DE PALABRA ######################################

#traigo todos los tokens ya generados para frases
tokk=tokRRSS+tokPQRS+tokENCUESTA
d=pd.DataFrame(tokk)

#Eliminto todos los que sean stopwords
index_sw=d[d[0].isin(stop_words)].index
for i in index_sw:
    tokk[i]='-'

dic_word=FreqDist(tokk)
del dic_word['-'] 

#Selecciono las primeras 70 palabras
lis_max=list(dic_word.keys())[0:30]

#Genero una tabla para asociar los comentarios a las words
df_word=BT_COMENTARIOS[['id','comentario']]
df_word['word']=0


df_word_last=df_word.copy()

#Asocio las words a los comentarios correspondientes
for i in range(0,len(lis_max)-1):
    try:
        index_w=df_word[df_word['comentario'].str.contains(lis_max[i])&(df_word['word']==0)].index
        aux = df_word.copy()
        aux.loc[index_w,'word']=lis_max[i]
        aux=aux[(aux['word']==lis_max[i])]
        df_word_last=pd.concat([aux,df_word_last])
    except:
        pass
df_word_last.drop_duplicates(inplace=True)
df_word_lastv=df_word_last[['id','word']]
df_word_lastv=df_word_lastv[df_word_lastv['word']!=0]
df_word_lastv.rename(columns={'id':'ID','word':'WORD'},inplace=True)


print("Termina generación de palabra",str(dt.datetime.today()-start_process))

######################## CARGA DE DATAFRAMES EN ORACLE #########################

#Ultima limipeza
BT_COMENTARIOS['comentario']=BT_COMENTARIOS['comentario_inicial']

#Agrego a la tabla todo lo que no modifique: ivr, messenger y customer
df_bt_comentarios=pd.concat([BT_COMENTARIOS,BT_inicial_partida])
                               
df_bt_comentarios=BT_COMENTARIOS[['id','interaccion_id','tema_id','comentario','wrapup','sentimiento','fecha_proceso']]
df_bt_comentarios['tema_id']=df_bt_comentarios['tema_id'].apply(lambda x: -99 if pd.isna(x) else x)
df_bt_comentarios['tema_id']=df_bt_comentarios['tema_id'].apply(lambda x: int(x))

df_bt_comentarios['sentimiento']=df_bt_comentarios['sentimiento'].apply(lambda x: -99 if pd.isna(x) else x)
df_bt_comentarios['sentimiento']=df_bt_comentarios['sentimiento'].apply(lambda x: float(x))

print("Termina ultimos arreglos",str(dt.datetime.today()-start_process))

########################### CARGA DE TABLAS ################################3

# #Tabla BT_COMENTARIOS con sentimientos y temas asignados
# object_columns = [c for c in df_bt_comentarios.columns[df_bt_comentarios.dtypes == 'object'].tolist()]
# dtyp = {c:sa.types.VARCHAR(1 if df_bt_comentarios[c].str.len().max()==0 else df_bt_comentarios[c].str.len().max() ) for c in object_columns} #Para que no tengn formato colab

# df_bt_comentarios.to_sql("bt_comentarios_temp", engine, schema='EFECTY_VOC', if_exists='replace',  index=False, chunksize=2000, dtype=dtyp)
# print("Termina carga BT_COMENTARIOS_TEMP",str(dt.datetime.today()-start_process))

# # Tabla que con los temas unicos y sus sentimientos. Tabla estática
# engine.execute(sa_text(F'TRUNCATE TABLE EFECTY_VOC.LK_TEMA').execution_options(autocommit=True))
# dtyp2 = {c:sa.types.VARCHAR(dic_cort[c].str.len().max()) for c in dic_cort.columns[dic_cort.dtypes == 'object'].tolist()} #Para que no tengn formato colab
# dic_cort.to_sql("lk_tema", engine, schema='EFECTY_VOC', if_exists='replace', index=False, chunksize=2000, dtype=dtyp2)
# print("Termina carga LK_TEMA",str(dt.datetime.today()-start_process))

# dtyp3 = {c:sa.types.VARCHAR(df_word_lastv[c].str.len().max()) for c in df_word_lastv.columns[df_word_lastv.dtypes == 'object'].tolist()} #Para que no tengn formato colab
# df_word_lastv.to_sql("bt_word", engine, schema='EFECTY_VOC', if_exists='replace',index=False, chunksize=2000, dtype=dtyp3)
# print("Termina carga bt_word",str(dt.datetime.today()-start_process))

# # Tabla con las key sentences
# dtyp4 = {c:sa.types.VARCHAR(df_sentence[c].str.len().max()) for c in df_sentence.columns[df_sentence.dtypes == 'object'].tolist()} #Para que no tengn formato colab
# df_sentence.to_sql("bt_frases", engine, schema='EFECTY_VOC', if_exists='replace', index=False, chunksize=2000, dtype=dtyp4)
# print("Termina carga BT_FRASES",str(dt.datetime.today()-start_process))

Termina conexión 0:00:00
Termina query 0:03:14.136039
La query mira desde 18 diciembre en adelante


<ipython-input-194-3a665f702e54>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BT_COMENTARIOS['comentario_inicial']=BT_COMENTARIOS['comentario']
<ipython-input-194-3a665f702e54>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BT_COMENTARIOS['comentario']=BT_COMENTARIOS['comentario'].astype(str)
C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

Termina trabajo sobre texto 0:03:14.521281


<ipython-input-194-3a665f702e54>:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BT_COMENTARIOS['sentimiento']=BT_COMENTARIOS['sentimientoV4'].apply(lambda x: 0.5 if x==1 else x)
<ipython-input-194-3a665f702e54>:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BT_COMENTARIOS['sentimiento']=BT_COMENTARIOS['sentimiento'].apply(lambda x: 1 if x==2 else x)
<ipython-input-194-3a665f702e54>:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

Termina generación sentimiento 0:10:00.149578
Termina generación tema 0:10:00.294915
frases y palabras solo mira mes actual, y lo filtrado arriba


<ipython-input-194-3a665f702e54>:329: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col1]=df[col1].apply(lambda x: re.sub('[\W]',' ',str(x))) #elimino todo lo que no es numero o letra
<ipython-input-194-3a665f702e54>:330: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col1]=df[col1].apply(lambda x: re.sub('\s\s',' ',str(x))) #elimino dbole espacio generado por lo de arriba
<ipython-input-194-3a665f702e54>:331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

Termina generación de frase 0:10:07.082043


<ipython-input-194-3a665f702e54>:557: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word['word']=0
<ipython-input-194-3a665f702e54>:583: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BT_COMENTARIOS['comentario']=BT_COMENTARIOS['comentario_inicial']


Termina generación de palabra 0:10:07.349655
Termina ultimos arreglos 0:10:07.373912


<ipython-input-194-3a665f702e54>:589: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bt_comentarios['tema_id']=df_bt_comentarios['tema_id'].apply(lambda x: -99 if pd.isna(x) else x)
<ipython-input-194-3a665f702e54>:590: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bt_comentarios['tema_id']=df_bt_comentarios['tema_id'].apply(lambda x: int(x))
<ipython-input-194-3a665f702e54>:592: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [196]:
lk_tema

NameError: name 'lk_tema' is not defined